In [13]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

import matplotlib.pyplot as plt
from PIL import Image

from mplsoccer import PyPizza, add_image, FontManager

#load fonts
font_normal = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Regular.ttf?raw=true"))
font_italic = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Italic.ttf?raw=true"))
font_bold = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                         "Roboto-Medium.ttf?raw=true"))

#Compared against
vs_Position = pageSoup.find_all("a", {"class": "sr_preset"})
vs_Position = vs_Position[0].text.strip()

#player name
plr = pageSoup.find_all("h1", {"itemprop": "name"})
plr = plr[0].text.strip()

#birthdate
players_open = pageSoup.find('div', attrs={'id':'info'})
players_open_birth = players_open.find_all('span',{"itemprop": "birthDate"})
players_open_club = players_open.find_all("a")

if len(players_open_club) > 3:
    tm = players_open_club[2].text.strip()
else:
    tm = players_open_club[1].text.strip()
birthdate = players_open_birth[0].text.strip()

#set League and Year
stats_pullout = pageSoup.find('div', attrs={'class':'stats_pullout'})
stats_pullout_year = stats_pullout.find("h4")
stats_pullout_lg = stats_pullout.find_all("strong")
stats_pullout_min = stats_pullout.find_all("p")

sn = stats_pullout_year.text.strip()
lg = stats_pullout_lg[0].text.strip()

minvar = len(stats_pullout.find_all("strong"))*2
min_played = stats_pullout_min[minvar].text.strip()

# parameter list
params = ["Shots Total",
"npxG",
"xA",
"SCA (PassLive)",
"Passes into\nPenalty Area",
"Pass\nCompletion %",
"Progressive\nPasses",
"Passes into\nFinal Third",
"Progressive Carries",
"Dribbles\nCompleted",
"Pressures",
"Successful\nPressures",
"Tackles",
"Interceptions",
"Ball Recoveries"
]

# value list
# The values are taken from the excellent fbref website (supplied by StatsBomb)
values = [int(Shots_Total),
int(npxG),
int(xA),
int(SCA_PassLive),
int(Passes_into_Penalty_Area),
int(Pass_Completion_Percent),
int(Progressive_Passes),
int(Passes_into_Final_Third),
int(Progressive_Carries),
int(Dribbles_Completed),
int(Pressures),
int(Successful_Pressures),
int(Tackles),
int(Interceptions),
int(Ball_Recoveries)
]

# color for the slices and text
slice_colors = ["#000034"] * 5 + ["#696969"] * 5 + ["#720000"] * 5
text_colors = ["#F2F2F2"] * 15

# instantiate PyPizza class
baker = PyPizza(
    params=params,                  # list of parameters
    background_color="#EBEBE9",     # background color
    straight_line_color="#EBEBE9",  # color for straight lines
    straight_line_lw=1,             # linewidth for straight lines
    last_circle_lw=0,               # linewidth of last circle
    other_circle_lw=1,              # linewidth for other circles
    other_circle_ls="-.",           # linestyle for other circles
    inner_circle_size=20            # size of inner circle
)

# plot pizza
fig, ax = baker.make_pizza(
    values,                          # list of values
    figsize=(16, 17),                # adjust figsize according to your need
    color_blank_space="same",        # use same color to fill blank space
    slice_colors=slice_colors,       # color for individual slices
    value_colors=text_colors,        # color for the value-text
    value_bck_colors=slice_colors,   # color for the blank spaces
    blank_alpha=0.3,                 # alpha for blank-space colors
    kwargs_slices=dict(
        edgecolor="#F2F2F2", zorder=2, linewidth=1
    ),                               # values to be used when plotting slices
    kwargs_params=dict(
        color="#000000", fontsize=18,
        fontproperties=font_normal.prop, va="center"
    ),                               # values to be used when adding parameter
    kwargs_values=dict(
        color="#000000", fontsize=20,
        fontproperties=font_normal.prop, zorder=3,
        bbox=dict(
            edgecolor="#000000", facecolor="cornflowerblue",
            boxstyle="round,pad=0.2", lw=1
        )
    )                                # values to be used when adding parameter-values
)

# add title
fig.text(
    0.515, 0.975, plr, size=35,
    ha="center", fontproperties=font_bold.prop, color="#000000"
)

# add subtitle
fig.text(
    0.515, 0.953,
    "Percentile Rank vs Top-Five League " + vs_Position + " | Season " + sn,
    size=18,
    ha="center", fontproperties=font_bold.prop, color="#000000"
)

# add credits
CREDIT_1 = "created by: @oh_that_crab"
CREDIT_2 = "data: StatsBomb via fbref"
CREDIT_3 = "inspired by: @Worville, @FootballSlices, @somazerofc & @Soumyaj15209314"

fig.text(
    0.99, 0.02, f"{CREDIT_1}\n{CREDIT_2}\n{CREDIT_3}", size=17,
    fontproperties=font_italic.prop, color="#000000",
    ha="right"
)

# add playting infor
Info_1 = "Cur Club: " + tm
Info_2 = lg
Info_3 = "Minutes Played: " + min_played
Info_4 = "Born: " + birthdate

fig.text(
    0.02, 0.02, f"{Info_1}\n{Info_2}\n{Info_3}\n{Info_4}", size=25,
    color="#000000",
    ha="left"
)

# add text Attacking
fig.text(
    0.35, 0.925, "Attacking", size=20,
    fontproperties=font_bold.prop, color="#000000"
)
# add text Possession
fig.text(
    0.5, 0.925, "Possession", size=20,
    fontproperties=font_bold.prop, color="#000000"
)
# add text Possession
fig.text(
    0.67, 0.925, "Defending", size=20,
    fontproperties=font_bold.prop, color="#000000"
)

                               

# add rectangles
fig.patches.extend([
    plt.Rectangle(
        (0.31, 0.9225), 0.025, 0.021, fill=True, color="#000034",
        transform=fig.transFigure, figure=fig
    ),
    plt.Rectangle(
        (0.462, 0.9225), 0.025, 0.021, fill=True, color="#696969",
        transform=fig.transFigure, figure=fig
    ),
    plt.Rectangle(
        (0.632, 0.9225), 0.025, 0.021, fill=True, color="#720000",
        transform=fig.transFigure, figure=fig
    ),
])


# add image
ax_image = add_image(
    cs_circ, fig, left=0.4478, bottom=0.4385, width=0.13, height=0.13
)   # these values might differ when you are plotting


plt.savefig(plr + " - " + sn + ' '+ vs_Position + ' slice' +'.png')
plt.show()

IndexError: single positional indexer is out-of-bounds

In [12]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

import matplotlib.pyplot as plt
from PIL import Image

from mplsoccer import PyPizza, add_image, FontManager

# grab the url from the src attribute of the embed code from fbref.com
url = "https://fbref.com/en/players/44bfb6c5/scout/10728/Ruben-Neves-Scouting-Report"

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
}
pageTree = requests.get(url, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

# Grab all tables from the webpage
dfs = pd.read_html(pageTree.content, header=[1])

# Check the number of tables and their shapes
print("Number of tables found:", len(dfs))
for i, df in enumerate(dfs):
    print(f"Shape of table {i}:", df.shape)

# Assuming you want the first table that was originally targeted
df = dfs[0]  # Update this index if the required table is not the first one

# Assuming data indices are still valid, check them
if 'Percentile' in df.columns:
    print("Percentile column found, proceeding with extraction...")
    try:
        # Your existing code to set stats values
        # ...
        Shots_from_free_kicks = df.iloc[21]['Percentile']  # Example index usage
    except IndexError as e:
        print(f"IndexError: {e} - Check if DataFrame has enough rows and correct indexing.")
else:
    print("Percentile column not found in the DataFrame.")


Number of tables found: 3
Shape of table 0: (21, 3)
Shape of table 1: (9, 5)
Shape of table 2: (169, 3)
Percentile column found, proceeding with extraction...
IndexError: single positional indexer is out-of-bounds - Check if DataFrame has enough rows and correct indexing.


In [11]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests

import matplotlib.pyplot as plt
from PIL import Image

from mplsoccer import PyPizza, add_image, FontManager

# grab the url from the src attribute of the embed code from fbref.com
url = "https://fbref.com/en/players/44bfb6c5/scout/10728/Ruben-Neves-Scouting-Report"

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
pageTree = requests.get(url, headers=headers)
pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

# Grab all tables from the webpage
dfs = pd.read_html(pageTree.content, header=[1])

# Use the table with more rows, assuming it's the one with comprehensive data
df = dfs[2]  # Change this index based on your requirement

# Print some sample data to confirm it's the correct table
print(df.head())  # Display the first few rows to check if the DataFrame looks correct

# Now process the DataFrame safely
try:
    # set stats values
    Shots_from_free_kicks = df.iloc[21]['Percentile']
    # Other statistics can be extracted similarly using safe indexing
except IndexError as e:
    print(f"IndexError: {e} - Check if DataFrame has enough rows and correct indexing.")

# Initialize font management for charts
font_normal = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Regular.ttf?raw=true"))
font_italic = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                           "Roboto-Italic.ttf?raw=true"))
font_bold = FontManager(("https://github.com/google/fonts/blob/main/apache/roboto/static/"
                         "Roboto-Medium.ttf?raw=true"))

# Extract player and team information
vs_Position = pageSoup.find_all("a", {"class": "sr_preset"})[0].text.strip()
plr = pageSoup.find_all("h1", {"itemprop": "name"})[0].text.strip()
players_open = pageSoup.find('div', attrs={'id':'info'})
players_open_birth = players_open.find_all('span',{"itemprop": "birthDate"})
players_open_club = players_open.find_all("a")
tm = players_open_club[2].text.strip() if len(players_open_club) > 3 else players_open_club[1].text.strip()
birthdate = players_open_birth[0].text.strip()

# Extract league and season information
stats_pullout = pageSoup.find('div', attrs={'class':'stats_pullout'})
stats_pullout_year = stats_pullout.find("h4")
stats_pullout_lg = stats_pullout.find_all("strong")
stats_pullout_min = stats_pullout.find_all("p")
sn = stats_pullout_year.text.strip()
lg = stats_pullout_lg[0].text.strip()
min_played = stats_pullout_min[len(stats_pullout.find_all("strong"))*2].text.strip()

# Prepare parameters and values for PyPizza plot
params = ["Shots Total", "npxG", "xA", "SCA (PassLive)", "Passes into Penalty Area", "Pass Completion %", "Progressive Passes", "Passes into Final Third", "Progressive Carries", "Dribbles Completed", "Pressures", "Successful Pressures", "Tackles", "Interceptions", "Ball Recoveries"]
values = [int(df.iloc[i]['Percentile']) for i in [15, 26, 50, 92, 53, 34, 55, 52, 156, 147, 121, 122, 110, 133, 181]]  # Adjust indices based on correct values

# Color settings for PyPizza
slice_colors = ["#000034"] * 5 + ["#696969"] * 5 + ["#720000"] * 5
text_colors = ["#F2F2F2"] * 15

# Create and plot PyPizza
baker = PyPizza(params=params, background_color="#EBEBE9", straight_line_color="#EBEBE9", straight_line_lw=1, last_circle_lw=0, other_circle_lw=1, other_circle_ls="-.", inner_circle_size=20)
fig, ax = baker.make_pizza(values, figsize=(16, 17), color_blank_space="same", slice_colors=slice_colors, value_colors=text_colors, value_bck_colors=slice_colors, blank_alpha=0.3, kwargs_slices=dict(edgecolor="#F2F2F2", zorder=2, linewidth=1), kwargs_params=dict(color="#000000", fontsize=18, fontproperties=font_normal.prop, va="center"), kwargs_values=dict(color="#000000", fontsize=20, fontproperties=font_normal.prop, zorder=3, bbox=dict(edgecolor="#000000", facecolor="cornflowerblue", boxstyle="round,pad=0.2", lw=1)))

# Add additional text and elements to plot
fig.text(0.515, 0.975, plr, size=35, ha="center", fontproperties=font_bold.prop, color="#000000")
fig.text(0.515, 0.953, "Percentile Rank vs Top-Five League " + vs_Position + " | Season " + sn, size=18, ha="center", fontproperties=font_bold.prop, color="#000000")

# Save and show plot
plt.savefig(plr + " - " + sn + ' '+ vs_Position + ' slice' +'.png')
plt.show()


            Statistic Per 90 Percentile
0               Goals   0.17         81
1             Assists   0.03         35
2     Goals + Assists   0.20         66
3   Non-Penalty Goals   0.07         58
4  Penalty Kicks Made   0.10         96


IndexError: list index out of range